In [ ]:
import matplotlib
%matplotlib inline
from matplotlib import pyplot
import numpy
import os
import datetime
# Create once the collection for this notebook
collection_id = None
import SEAScope.lib.utils
SEAScope.lib.utils.init_ids(10, 1000)

## Load and select data
<span style="color:green">**Select the surface velocity from GlobCurrent geostrophic collection you want to play with (current norm raster)**<br>
**Draw a polygon on SEAScope**<br> 
**Click on the extract button on the polygon window**<br>
</span>


In [ ]:
# Load data directly from viewer memory
from SEAScope.lib import get_extracted_data
extractions = get_extracted_data()
granule_id = None

In [ ]:
#from SEAScope.lib import load_pyo
#data_path = ''
#extractions = load_pyo(data_path)
## Print a list of path of extracted granules 
print('\n'.join(extractions.keys()))

**Print selected granules**

In [ ]:
for k, data in enumerate(extractions.keys()):
    print('{} - {}'.format(k, os.path.basename(data)))
    print('\n'.join(['\t{}'.format(x) for x in extractions[data]['data']]))
    print(extractions[data]['meta']['start'])

## Play with the data
** Compute trajectory of fictive swell ray at initial position (x0, y0) using the dispertion relation in presence of surface current : <br>
$ \omega_c = \omega+ \vec{k}.\vec{u}$<br> 
$ \frac{dx}{dt}=\frac{d\omega_c}{dk}$<br> 
$ \frac{dk}{dt}=-\frac{d\omega_c}{d}$<br> **


<span style="color:red">** Fill variable name for your selected velocity **</span>

In [ ]:
# Get longitudes, latitudes from gcps and time from first granule
granule_uri =  next( v for i, v in enumerate(extractions.keys()) if i == 0)
extraction = extractions[granule_uri]
start = extraction['meta']['start']
print(extraction['meta']['fields'])

In [ ]:
name_var_u = 'ugos'
name_var_v = 'vgos'

extraction = extractions[granule_uri]   
u = extraction['data'][name_var_u]
v = extraction['data'][name_var_v]

from SEAScope.lib.utils import get_lonlat
lon2D, lat2D = get_lonlat(extraction, numpy.shape(u))


In [ ]:
velocity = numpy.sqrt(u**2 + v**2)
velocity.shape

In [ ]:
fig = pyplot.figure(figsize = (16,10))
pyplot.imshow(numpy.flipud(velocity), interpolation='bicubic', cmap='jet', vmin=0 , vmax=2)
cbar = pyplot.colorbar()
X, Y = numpy.meshgrid(numpy.arange(1, u.shape[1]+1), numpy.arange(1, u.shape[0]+1))
print(X.shape, u.shape)
Q = pyplot.quiver(X[::15, ::15], Y[::15, ::15],numpy.clip(numpy.flipud(u[::15,::15]),-2,2), numpy.clip(numpy.flipud(v[::15,::15]),-2,2))
qk = pyplot.quiverkey(Q, 0.5, 0.98, 2, r'$2 \frac{m}{s}$', labelpos='W',
                   fontproperties={'weight': 'bold'})
#plt.xlim(40, 150)
#plt.ylim(180, 120)

In [ ]:
res = (lat2D[1,0] - lat2D[0,0]) * 100000
res = (lat2D[int(u.shape[0]/2 + 1), int(u.shape[1]/2)]
       - lat2D[int(u.shape[0]/2), int(u.shape[1]/2)]) * 100000
'resolution is {} m'.format(res)

<span style="color:red"> ** Play with nstep (number of time steps) and nseed (number of rays) ** </span>

In [ ]:
# Bilinear interpolation
def ev(var, x, y):
    if numpy.isnan(x) or numpy.isnan(y):
        return numpy.nan
    nx, ny = numpy.shape(var)
    i = int(x)
    j = int(y)
    if (i > nx - 2) or (j > ny - 2):
        return numpy.nan
    dx = x - i
    dy = y - j
    uix = dx * var[i + 1, j] + (1 - dx) * var[i, j]
    uiy = dx * var[i + 1, j + 1] + (1 - dx) * var[i, j + 1]
    ui = dy * uiy + (1 - dy) * uix
    return ui

# Initialize wave ray position
granule_id = None
# Time step for advection (in sec)
dt = 3600/3
# Total time of advection 
nstep = 50 #! Change this
# Number of seeds
nseed = 70 #! Change this
# Wave number
k = 2 * numpy.pi/250
# Wave direction in radians
teta0 = numpy.deg2rad(55)
# Starting line
l0 = u.shape[0] - 50
# Starting column
c0 = 50

res = (lat2D[int(u.shape[0]/2 + 1), int(u.shape[1]/2)]
       -lat2D[int(u.shape[0]/2), int(u.shape[1]/2)]) * 100000
x = numpy.full((nstep+1, nseed), numpy.nan)
y = numpy.full((nstep+1, nseed), numpy.nan)
lon = numpy.full((nstep+1, nseed), numpy.nan)
lat = numpy.full((nstep+1, nseed), numpy.nan)
for c in range(nseed):
    x[0, c] = -l0
    y[0, c] = c0 + 5*c
    lon[0, c] = ev(lon2D, x[0, c], y[0, c])
    lat[0, c] = ev(lat2D, x[0, c], y[0, c])
    teta = teta0
    kx = k * numpy.sin(teta)
    ky = k * numpy.cos(teta)
    for i in range(nstep):
        if x[i, c] is numpy.ma.masked or y[i, c] is numpy.ma.masked:
            continue
        omega = numpy.sqrt(9.81 * (kx**2+ky**2)) + (kx*ev(u,x[i, c], y[i, c])
                           + ky * ev(v, x[i, c], y[i, c]))
        dx = dt * (ev(u, x[i, c], y[i, c]) + numpy.sqrt(9.81/(numpy.sqrt(kx**2+ky**2)))
                 * numpy.sin(teta))/res
        dy = dt * (ev(v, x[i, c], y[i, c]) + numpy.sqrt(9.81/(numpy.sqrt(kx**2+ky**2)))
                 *numpy.cos(teta))/res
        teta = numpy.arctan(dx/dy)
        if dx is numpy.ma.masked or dy is numpy.ma.masked:
            continue
        domegax = omega - (numpy.sqrt(9.81 * (kx**2+ky**2))
                           + (kx * ev(u, x[i, c] + dx, y[i, c])
                           + ky * ev(v, x[i, c] + dx, y[i, c])))
        domegay = omega - (numpy.sqrt(9.81 * (kx**2+ky**2))
                           + (kx * ev(u, x[i, c], y[i, c] + dy)
                           + ky * ev(v, x[i, c], y[i, c] + dy)))

        x[i + 1, c] = x[i, c] + dx
        y[i + 1, c] = y[i, c] + dy
        lon[i + 1, c] = ev(lon2D, x[i + 1, c], y[i + 1, c])
        lat[i + 1, c] = ev(lat2D, x[i + 1, c], y[i + 1, c])
        dkx = dt * domegax/dx/25000
        dky = dt * domegay/dy/25000
        kx = kx + dkx
        ky = ky + dky

In [ ]:
fig = pyplot.figure(figsize = (18,10))
pyplot.imshow(numpy.flipud(velocity),interpolation='bicubic',cmap='jet', vmin=0 , vmax=2)
cbar=pyplot.colorbar()
Q = pyplot.quiver(X[::15, ::15], Y[::15, ::15],
               numpy.clip(numpy.flipud(u[::15, ::15]), -2, 2),
               numpy.clip(numpy.flipud(v[::15, ::15]), -2, 2))
qk = pyplot.quiverkey(Q, 0.5, 0.98, 2, r'$2 \frac{m}{s}$', labelpos='W',
                   fontproperties={'weight': 'bold'})
for c in range(nseed):
    pyplot.plot(y[:, c],-x[:, c], color='w', linewidth=1.5)


## Export wave rays to SEAScope

**Define Collection:** <br>
** A granule has to belong to a collection **<br>

In [ ]:
from SEAScope.lib.utils import create_collection, init_ids
# Make sure to keep the collection identifier, we will need it later
collection_id, collection = create_collection('User - Wave rays')
print(collection_id)

# IP address and port used to reach the SEAScope standalone application
host = '127.0.0.1'
port = 11155

import SEAScope.upload
with SEAScope.upload.connect(host, port) as link:
# Send create collection job
    SEAScope.upload.collection(link, collection)

**Upload granule to SEAScope: <br>
Define shell of granule, create granule <br>
The granule will contain each matrix of modified data as what we call a "field"<br>**
<span style="color:red">** Enter a name for each field (field_name_time) **</span></br>
**Upload collection and granule to SEAScope**

In [ ]:
#import sys
#import logging
#logger = logging.getLogger()
#handler = logging.StreamHandler(stream=sys.stdout)
#handler.setLevel(logging.DEBUG)
#logger.addHandler(handler)
#logger.setLevel(logging.DEBUG)
#logger.info('Logging set')


In [ ]:
SEAScope.upload.throttler.min_delay = 50

In [ ]:
from SEAScope.lib.utils import create_granule
# Shell of Granule
# Each position of the trajectory will be used as GCP
for c in range(nseed):
    gcps = []
    for i in range(0, nstep+1):
        if ~numpy.isnan(lon[i,c]) :
            gcp = {'lon': lon[i,c], 'lat': lat[i,c], 'i': i, 'j': 0}
            gcps.append(gcp)

# Create granule
    stop_dt = start + datetime.timedelta(days=7)
    granule_id, granule = create_granule(collection_id, gcps, start, stop_dt)

# Set variables
# Enter a name for each variable field
    from SEAScope.lib.utils import set_field
    field_name_time ='time'
    set_field(granule, field_name_time, numpy.arange(nstep+1))

    # Send create granule job
    with SEAScope.upload.connect(host, port) as link:   
        SEAScope.upload.granule(link, granule)

**Telling SEAScope what to display: <br>
 The next step is variables definition. ** <br>
**<span style="color:red"> Enter a name for each variable you want to display (variable_name_time, ...) </span>**</br>

In [ ]:
from SEAScope.lib.utils import create_variable

# Create a variable for each parameter
variable_name_time = 'time'
var1 = create_variable(collection, variable_name_time, [field_name_time], dims=1)

# Upload to SEAScope
with SEAScope.upload.connect(host, port) as link:
    # Send create variable jobs
    SEAScope.upload.variable(link, var1)

** Telling SEAScope how to display variables: <br>
The rendering options are defined in the following cell. <br>
<span style="color:red"> You can play with min, max values, the colormap ... and check how the rendering changes on SEAScope </span>
**

In [ ]:
# Rendering configuration for the time
rcfg1 = var1['rendering']
rcfg1['min'] = 1
rcfg1['max'] = 50
rcfg1['colormap'] = 'jet'
rcfg1['zindex'] = .923
rcfg1['opacity'] = 0.8



# Upload to SEAScope
with SEAScope.upload.connect(host, port) as link:
    # Send create rendering config jobs
    SEAScope.upload.rendering_config(link, rcfg1)

<span style="color:green">** Go back to SEAScope and display the User Wave ray collection on top of the Globcurrent geostrophy **</span>